In [4]:
import pickle

follower = {}
friend = {}
e_tweet = {}
m_tweet = {}
r_tweet = {}

with open('./Unfollower/15weeks_follower_dict.pkl', 'rb') as f:
    follower = pickle.load(f)

with open('./Unfollower/15weeks_friend_dict.pkl', 'rb') as f:
    friend = pickle.load(f)
    
with open('./Unfollower/e_tweet_dict.pkl', 'rb') as f:
    e_tweet = pickle.load(f)
    
with open('./Unfollower/m_tweet_dict.pkl', 'rb') as f:
    m_tweet = pickle.load(f)
    
with open('./Unfollower/r_tweet_dict.pkl', 'rb') as f:
    r_tweet = pickle.load(f)

In [24]:
# i = 0
# for key in m_tweet.keys():
#     if i < 100:
#         print(key)
#         print(len(m_tweet[key]))
#         for j in m_tweet[key]:
#             print(j)
        
#     i+=1
    
# print(i)

print(m_tweet["train"][1])


{'705367525983256577': {'374900083': ['@FRANKIEmusic @weareOneNight WOW WOW']}, '121220255': {'1950362468': ['@MissaFT 😲']}, '16555722': {'14983009': ['@G33KatWork @FauthNiklas waste of slush']}, '3194344953': {'3194344953': ['The best #DataScience students of the world are still actively competing on the @datasciencegame online challenge !… https://t.co/upb2RmTIA1']}, '2532451142': {'217313388': ['@Hacker_Horse @OpenZFS @eurobsdcon Wish you the best of luck :3'], '1588611': ["@Hacker_Horse @isotopp @recursive @darixzen Now, let's see the following fact:\n\nIs bash POSIX?\nNo.\n\nTherefore assum… https://t.co/jvubxTRt13", '@Hacker_Horse @isotopp @recursive @darixzen Quoting from man pages that Linux people don\'t know how to read:\n\n"env… https://t.co/r60ivnKLJY'], '3298570644': ["@Hacker_Horse I thought I'm the only one who have the same opinion :P\n\nReally unpopular.", '@Hacker_Horse Oh yeah, https://t.co/cnkkmRzIfY', '@Hacker_Horse I wonder what RMS thinks about this.', "@Hacker_H